In [ ]:
import torch
import numpy as np
from PIL import Image
import os
import gnss_lib_py as glp
import pandas as pd

from nemo.util import parse_colmap_point_cloud, elevation_function
from nemo.plotting import plot_3d_points, plot_surface
from nemo.dem import DEM

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

%load_ext autoreload
%autoreload 2

# KT-22

### USGS Elevation data

In [ ]:
# KT22 lat lon
lat = 39.182745
lon = -120.241651
alt = 2423.0

# Grid of lat lon samples
N = 64
delta = 0.05  # degrees
lat_range = np.linspace(lat - delta, lat + delta, N)
lon_range = np.linspace(lon - delta, lon + delta, N)
grid = np.meshgrid(lat_range, lon_range)
samples = np.vstack([grid[0].ravel(), grid[1].ravel()]).T

In [ ]:
# coordinates with known elevation 
lat_samples = samples[:, 0]
lon_samples = samples[:, 1]
   
# create data frame
df = pd.DataFrame({
    'lat': lat_samples,
    'lon': lon_samples
})

elevation_function(df, 'lat', 'lon')
df.head()

In [ ]:
# Convert lat long to NED
lla = df.values.astype(float)
local_frame = glp.LocalCoord.from_geodetic(np.array([[lat, lon, alt]]))
ned = local_frame.geodetic_to_ned(lla.T)

In [ ]:
x, y, z = ned
x = x.reshape(N, N)
y = y.reshape(N, N)
z = z.reshape(N, N)

# Plot the surface
fig = plot_surface(x, y, z)
fig.show()

# Red Rocks

### Imagery EXIF geo location

In [ ]:
img_folder = '../../Raw_data/DroneMapper/DroneMapper-RedRocks-Oblique'

def get_lla_from_exif(exif_data):
    lat = exif_data[34853][2][0] + exif_data[34853][2][1] / 60.0 + exif_data[34853][2][2] / 3600.0
    lon = exif_data[34853][4][0] + exif_data[34853][4][1] / 60.0 + exif_data[34853][4][2] / 3600.0
    alt = exif_data[34853][6]
    return lat, lon, alt

lla = []

# Loop through images in folder
for img in os.listdir(img_folder):
    img_path = os.path.join(img_folder, img)
    im = Image.open(img_path)
    exif_data = im._getexif()
    lat, lon, alt = get_lla_from_exif(exif_data)
    lla.append([lat, lon, alt])

lla = np.array(lla, dtype=np.float32)

In [ ]:
local_frame = glp.LocalCoord.from_geodetic(lla[0][:,None])
ned = local_frame.geodetic_to_ned(lla.T)

In [ ]:
fig = plot_3d_points(x=ned[0], y=ned[1], z=ned[2])
fig.show()

### COLMAP camera poses

In [ ]:
import json

# Path to transforms.json
with open('../../nerfstudio/data/RedRocks/transforms.json', 'r') as f:
    data = json.load(f)

# Extract camera poses
poses = np.array([[data['frames'][i]['transform_matrix']] 
                        for i in range(len(data['frames']))])
poses = np.squeeze(poses)
positions = poses[:, :3, 3]

fig = plot_3d_points(x=positions[:,0], y=positions[:,1], z=positions[:,2])
fig.show()

### COLMAP sparse point cloud

In [ ]:
path = '../data/redrocks/redrocks_points3D.txt'
points, colors = parse_colmap_point_cloud(path)

In [ ]:
# Extract the points within range xyz in [-5, 5]
points = points[(points[:,0] > -5) & (points[:,0] < 5)]
points = points[(points[:,1] > -5) & (points[:,1] < 5)]
points = points[(points[:,2] > -5) & (points[:,2] < 5)]

fig = plot_3d_points(x=points[:,0], y=points[:,1], z=points[:,2], markersize=1)
fig.show()

Above points are in COLMAP frame $F_{colmap}$. We want to transform to the NeRF scene coordinate frame $F_{nerf}$. 
The transformations are as follows:
1. Manual camera plane alignment rotation from `align_camera_plane.ipynb` (in nerfstudio repo)
2. Nerfstudio dataparser scale and transform, stored in `dataparser_transforms.json` (in outputs folder)

In [ ]:
# For Red Rocks
manual_R = np.array([[-0.25241505,  0.96618594, -0.0526439],
                     [-0.69407693, -0.21869781, -0.68587789],
                     [-0.67419868, -0.13658698,  0.72580999]])
dataparser_T = np.array([[0.0, 1.0, 0.0, 0.02047962136566639],
                         [1.0, 0.0, 0.0, -0.17118817567825317],
                         [0.0, 0.0, -1.0, 0.10579380393028259]])
dataparser_scale = 0.21856701094642245

In [ ]:
points_tf = points @ manual_R.T
points_tf = points_tf @ dataparser_T[:3,:3].T
points_tf = points_tf + dataparser_T[:3,3]
points_tf = points_tf * dataparser_scale

# Swap x and z axes
points_tf = points_tf @ np.array([[0, 0, 1], [0, 1, 0], [-1, 0, 0]])

In [ ]:
fig = plot_3d_points(x=points_tf[:,0], y=points_tf[:,1], z=points_tf[:,2], color=points_tf[:,2], markersize=1)
fig.show()
# NOTE: looks like incorrect frame (X up instead of Z up)

### NEMo height field

In [ ]:
from nemo.nemo import Nemo

nemo = Nemo()
nemo.load_weights('../models/redrocks_encs_relu.pth', '../models/redrocks_heightnet_relu.pth')

N = 64
# xmin, xmax, ymin, ymax
bounds = (-0.3, 0.8, -0.45, 0.5) # red rocks
#bounds = (-0.75, 0.75, -0.75, 0.75) # kt22
xs = torch.linspace(bounds[0], bounds[1], N, device=device)
ys = torch.linspace(bounds[2], bounds[3], N, device=device)
XY_grid = torch.meshgrid(xs, ys, indexing='xy')
XY_grid = torch.stack(XY_grid, dim=-1)
positions = XY_grid.reshape(-1, 2)

heights = nemo.get_heights(positions)

z_grid = heights.reshape(N, N).detach().cpu().numpy()
x_grid = XY_grid[:,:,0].detach().cpu().numpy()
y_grid = XY_grid[:,:,1].detach().cpu().numpy()

fig = plot_surface(x_grid, y_grid, z_grid, fig=fig)
fig.show()

### DroneMapper DEM

In [ ]:
from osgeo import gdal

# Open the GeoTIFF file
# (checked by seems vertical datum metadata is missing)
dataset = gdal.Open('../data/redrocks/DEM32-DroneMapper.tif')

if dataset is None:
    print("Failed to open the GeoTIFF file")
    exit(1)

# Read raster data
band = dataset.GetRasterBand(1)  # Assuming it's a single-band raster
elevation_data = band.ReadAsArray()  

# Use geotransform parameters to calculate extent (use a local ENU coordinate frame with origin at center of DEM)
geotransform = dataset.GetGeoTransform()
top_left_x, pixel_width, x_rotation, top_left_y, y_rotation, pixel_height = geotransform
cols = dataset.RasterXSize
rows = dataset.RasterYSize

# Calculate East and North extents
east_extent = cols * pixel_width
north_extent = rows * abs(pixel_height)

In [ ]:
dem = DEM(elevation_data, extent=(east_extent, north_extent))

In [ ]:
dem_ds = dem.downsample(10)
dem.handle_missing_data()
dem_ds.show()

---
